In [50]:
import pandas as pd
import re

# Pré-processamento dos Dados

### Carregar Base de dados

In [51]:
laptop_price = pd.read_csv("../datasets/raw/laptop_price.csv", encoding="ISO-8859-1")
laptop_price.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


### Verificar tipo das colunas e dropar coluna laptop_id

In [52]:
laptop_price = laptop_price.rename(columns={'ScreenResolution': 'screen_resolution'})
laptop_price.columns = laptop_price.columns.str.lower()
laptop_price = laptop_price.drop(columns='laptop_id')
laptop_price.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   company            1303 non-null   object 
 1   product            1303 non-null   object 
 2   typename           1303 non-null   object 
 3   inches             1303 non-null   float64
 4   screen_resolution  1303 non-null   object 
 5   cpu                1303 non-null   object 
 6   ram                1303 non-null   object 
 7   memory             1303 non-null   object 
 8   gpu                1303 non-null   object 
 9   opsys              1303 non-null   object 
 10  weight             1303 non-null   object 
 11  price_euros        1303 non-null   float64
dtypes: float64(2), object(10)
memory usage: 904.4 KB


### Verificar quantidade de valores únicos

In [53]:
print("Quantidade de Valores únicos por coluna")
for column in laptop_price.columns:
    print(f"{column}: {len(laptop_price[column].unique())}")

Quantidade de Valores únicos por coluna
company: 19
product: 618
typename: 6
inches: 18
screen_resolution: 40
cpu: 118
ram: 9
memory: 39
gpu: 110
opsys: 9
weight: 179
price_euros: 791


### Obter fabricante da cpu e gpu

In [54]:
laptop_price['cpu_manufacturer'] = laptop_price.cpu.str.split(n=1, expand=True)[0]
laptop_price['gpu_manufacturer'] = laptop_price.gpu.str.split(n=1, expand=True)[0]

### Tratar memory

In [55]:
laptop_price.memory.replace(to_replace = '([.][0])|([G][B])', value = '', regex=True, inplace=True)
laptop_price.memory.replace(to_replace = '([T][B])', value = '000', regex=True, inplace=True)
laptop_price.memory.replace(to_replace = '( )+', value = ' ', regex=True, inplace=True)

laptop_price['hdd'] = 0
laptop_price['ssd'] = 0
laptop_price['hybrid'] = 0
laptop_price['flash_storage'] = 0

### Verificar valores únicos

In [56]:
laptop_price.memory.unique()

array(['128 SSD', '128 Flash Storage', '256 SSD', '512 SSD', '500 HDD',
       '256 Flash Storage', '1000 HDD', '32 Flash Storage',
       '128 SSD + 1000 HDD', '256 SSD + 256 SSD', '64 Flash Storage',
       '256 SSD + 1000 HDD', '256 SSD + 2000 HDD', '32 SSD', '2000 HDD',
       '64 SSD', '1000 Hybrid', '512 SSD + 1000 HDD', '1000 SSD',
       '256 SSD + 500 HDD', '128 SSD + 2000 HDD', '512 SSD + 512 SSD',
       '16 SSD', '16 Flash Storage', '512 SSD + 256 SSD',
       '512 SSD + 2000 HDD', '64 Flash Storage + 1000 HDD', '180 SSD',
       '1000 HDD + 1000 HDD', '32 HDD', '1000 SSD + 1000 HDD',
       '512 Flash Storage', '128 HDD', '240 SSD', '8 SSD', '508 Hybrid',
       '512 SSD + 1000 Hybrid', '256 SSD + 1000 Hybrid'], dtype=object)

In [57]:
for i, row in enumerate(laptop_price['memory']):
    row = row.lower()
    for items in row.split('+ '):
        valores = items.split(' ', 1)
        if 'flash storage' in valores[1]:
            laptop_price.loc[i, 'flash_storage'] += eval(valores[0])
        elif 'hdd' in valores[1]:
            laptop_price.loc[i, 'hdd'] += eval(valores[0])
        elif 'ssd' in valores[1]:
            laptop_price.loc[i, 'ssd'] += eval(valores[0])
        elif 'hybrid' in valores[1]:
            laptop_price.loc[i, 'hybrid'] += eval(valores[0])

laptop_price.drop(columns=['memory'], inplace=True)

In [58]:
laptop_price.head()

,company,product,typename,inches,screen_resolution,cpu,ram,gpu,opsys,weight,price_euros,cpu_manufacturer,gpu_manufacturer,hdd,ssd,hybrid,flash_storage
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,Intel,Intel,0,128,0,0
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,Intel HD Graphics 6000,macOS,1.34kg,898.94,Intel,Intel,0,0,0,128
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,Intel HD Graphics 620,No OS,1.86kg,575.00,Intel,Intel,0,256,0,0
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,AMD Radeon Pro 455,macOS,1.83kg,2537.45,Intel,AMD,0,512,0,0
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60,Intel,Intel,0,256,0,0


### Remover GB em Ram

In [59]:
laptop_price.ram.replace(to_replace = '([G][B])', value = '', regex=True, inplace=True)

### Filtrar screen_resolution

In [60]:
laptop_price.screen_resolution = laptop_price.screen_resolution.str.extract(r'([0-9]+[x][0-9]+)')

### Filtrar Coluna Weight

In [61]:
laptop_price.weight = laptop_price.weight.str.slice(stop=-2)

### Valores máximos e minimos

In [62]:
laptop_price.describe()

,inches,price_euros,hdd,ssd,hybrid,flash_storage
count,1303.000000,1303.000000,1303.000000,1303.000000,1303.000000,1303.000000
mean,15.017191,1123.686992,413.783576,183.732924,8.831926,4.555641
std,1.426304,699.009043,515.818779,186.966903,92.567185,30.274090
min,10.100000,174.000000,0.000000,0.000000,0.000000,0.000000
25%,14.000000,599.000000,0.000000,0.000000,0.000000,0.000000
50%,15.600000,977.000000,0.000000,256.000000,0.000000,0.000000
75%,15.600000,1487.880000,1000.000000,256.000000,0.000000,0.000000
max,18.400000,6099.000000,2000.000000,1024.000000,1000.000000,512.000000


### Fazer downcasting das colunas

In [63]:
laptop_price.head()

,company,product,typename,inches,screen_resolution,cpu,ram,gpu,opsys,weight,price_euros,cpu_manufacturer,gpu_manufacturer,hdd,ssd,hybrid,flash_storage
0,Apple,MacBook Pro,Ultrabook,13.3,2560x1600,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,Intel,Intel,0,128,0,0
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,1.34,898.94,Intel,Intel,0,0,0,128
2,HP,250 G6,Notebook,15.6,1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,1.86,575.00,Intel,Intel,0,256,0,0
3,Apple,MacBook Pro,Ultrabook,15.4,2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,1.83,2537.45,Intel,AMD,0,512,0,0
4,Apple,MacBook Pro,Ultrabook,13.3,2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,Intel,Intel,0,256,0,0


In [65]:
items = {
    'ram': 'ram_gb',
    'hdd': 'hdd_gb',
    'ssd': 'ssd_gb',
    'hybrid': 'hybrid_gb',
    'flash_storage': 'flash_storage_gb',
    'weight': 'weight_kg'
}
laptop_price.rename(columns=items, inplace=True)

In [66]:
def downcasting(df, columns, type):
    for column in columns:
        df[column] = df[column].astype(type)
    
items = {
    'uint8': ['ram_gb'],
    'int16': ['hdd_gb', 'ssd_gb', 'hybrid_gb', 'flash_storage_gb'],
    'float32': ['weight_kg', 'inches'],
    'float64': ['price_euros'],
    'category': ['company', 'typename', 'screen_resolution', 'cpu', 'gpu', 'opsys', 'product', 'cpu_manufacturer', 'gpu_manufacturer']
}
    
for tipo, coluna in items.items():
    downcasting(laptop_price, coluna, tipo)    

### Ver Redução de memoria

In [67]:
laptop_price.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   company            1303 non-null   category
 1   product            1303 non-null   category
 2   typename           1303 non-null   category
 3   inches             1303 non-null   float32 
 4   screen_resolution  1303 non-null   category
 5   cpu                1303 non-null   category
 6   ram_gb             1303 non-null   uint8   
 7   gpu                1303 non-null   category
 8   opsys              1303 non-null   category
 9   weight_kg          1303 non-null   float32 
 10  price_euros        1303 non-null   float64 
 11  cpu_manufacturer   1303 non-null   category
 12  gpu_manufacturer   1303 non-null   category
 13  hdd_gb             1303 non-null   int16   
 14  ssd_gb             1303 non-null   int16   
 15  hybrid_gb          1303 non-null   int16   
 16  flash_

In [68]:
ordem = ['company', 
        'product', 
        'typename', 
        'inches', 
        'screen_resolution', 
        'cpu_manufacturer', 
        'cpu',
        'gpu_manufacturer', 
        'gpu', 
        'ram_gb', 
        'hdd_gb', 
        'ssd_gb', 
        'hybrid_gb', 
        'flash_storage_gb',
        'opsys',
        'weight_kg',
        'price_euros'
]

laptop_price = laptop_price[ordem]

In [69]:
laptop_price.head()

,company,product,typename,inches,screen_resolution,cpu_manufacturer,cpu,gpu_manufacturer,gpu,ram_gb,hdd_gb,ssd_gb,hybrid_gb,flash_storage_gb,opsys,weight_kg,price_euros
0,Apple,MacBook Pro,Ultrabook,13.3,2560x1600,Intel,Intel Core i5 2.3GHz,Intel,Intel Iris Plus Graphics 640,8,0,128,0,0,macOS,1.37,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel,Intel Core i5 1.8GHz,Intel,Intel HD Graphics 6000,8,0,0,0,128,macOS,1.34,898.94
2,HP,250 G6,Notebook,15.6,1920x1080,Intel,Intel Core i5 7200U 2.5GHz,Intel,Intel HD Graphics 620,8,0,256,0,0,No OS,1.86,575.00
3,Apple,MacBook Pro,Ultrabook,15.4,2880x1800,Intel,Intel Core i7 2.7GHz,AMD,AMD Radeon Pro 455,16,0,512,0,0,macOS,1.83,2537.45
4,Apple,MacBook Pro,Ultrabook,13.3,2560x1600,Intel,Intel Core i5 3.1GHz,Intel,Intel Iris Plus Graphics 650,8,0,256,0,0,macOS,1.37,1803.60


### Salvar para Pickle

In [70]:
laptop_price.to_pickle("../datasets/processed/laptop_price.pkl")